## *Recommandation des stations velib dans le 1er arrondissement de Paris*

In [1]:
%%init_spark

In [2]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://lenovo-ideapad.home:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1572902045469)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4493afb9


In [3]:
spark.version

res1: String = 2.4.4


In [4]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrameNaFunctions
import org.apache.spark.sql.types._
import scala.util.parsing.json._

import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrameNaFunctions
import org.apache.spark.sql.types._
import scala.util.parsing.json._


### 1- Chargement des données

In [5]:
 val velib: DataFrame = spark
      .read
      .option("header", true) // utilise la première ligne du (des) fichier(s) comme header
      .option("inferSchema", "true") // pour inférer le type de chaque colonne (Int, String, etc.)
      .option("delimiter", ";")
      .csv("/home/parfait/Documents/MS BGD/Data Mining_SD701/Projet_velib/data/velib-dispo-2810-matin.csv")


velib: org.apache.spark.sql.DataFrame = [Nombre de bornes disponibles: int, Nombre vélo en PARK+1: int ... 19 more fields]


In [6]:
println(s"Nombre de lignes : ${velib.count}")
println(s"Nombre de colonnes : ${velib.columns.length}")

Nombre de lignes : 1390
Nombre de colonnes : 21


In [7]:
velib.printSchema()

root
 |-- Nombre de bornes disponibles: integer (nullable = true)
 |-- Nombre vélo en PARK+1: integer (nullable = true)
 |-- Nombres de bornes en station: integer (nullable = true)
 |-- PARK + activation: string (nullable = true)
 |-- densityLevel: integer (nullable = true)
 |-- Achat possible en station (CB): string (nullable = true)
 |-- Description station: string (nullable = true)
 |-- maxBikeOverflow: integer (nullable = true)
 |-- Etat du Totem: string (nullable = true)
 |-- nbFreeDock: integer (nullable = true)
 |-- Nombre de vélo mécanique: integer (nullable = true)
 |-- PARK +: string (nullable = true)
 |-- nbDock: integer (nullable = true)
 |-- Nombre vélo électrique: integer (nullable = true)
 |-- Nombre vélo en PARK+14: integer (nullable = true)
 |-- Code de la station: integer (nullable = true)
 |-- Nom de la station: string (nullable = true)
 |-- Etat des stations: string (nullable = true)
 |-- Type de stations: string (nullable = true)
 |-- geo: string (nullable = true)


In [8]:
 val adresses: DataFrame = spark
      .read
      .option("header", true) // utilise la première ligne du (des) fichier(s) comme header
      .option("inferSchema", "true") // pour inférer le type de chaque colonne (Int, String, etc.)
      .option("delimiter", ";")
      .csv("/home/parfait/Documents/MS BGD/Data Mining_SD701/Projet_velib/data/adresses-75.csv")

adresses: org.apache.spark.sql.DataFrame = [id: string, id_fantoir: string ... 18 more fields]


In [9]:
println(s"Nombre de lignes : ${adresses.count}")
println(s"Nombre de colonnes : ${adresses.columns.length}")

Nombre de lignes : 151430
Nombre de colonnes : 20


In [10]:
adresses.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_fantoir: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- rep: string (nullable = true)
 |-- nom_voie: string (nullable = true)
 |-- code_postal: integer (nullable = true)
 |-- code_insee: integer (nullable = true)
 |-- nom_commune: string (nullable = true)
 |-- code_insee_ancienne_commune: string (nullable = true)
 |-- nom_ancienne_commune: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- alias: string (nullable = true)
 |-- nom_ld: string (nullable = true)
 |-- libelle_acheminement: string (nullable = true)
 |-- nom_afnor: string (nullable = true)
 |-- source_position: string (nullable = true)
 |-- source_nom_voie: string (nullable = true)



### 2- Cleaning data velib et adresses

In [11]:
val velibClean: DataFrame =  velib
    .withColumn("x_velib", split($"geo", ",").getItem(1).cast("double"))
    .withColumn("y_velib", split($"geo", ",").getItem(0).cast("double"))
    .withColumnRenamed("Nombre de bornes disponibles","nbre_bornes_dispo")
    .withColumnRenamed("Nombres de bornes en station","nbre_bornes_station")
    .withColumnRenamed("Achat possible en station (CB)","achat_cb")
    .withColumnRenamed("Description station","description")
    .withColumnRenamed("Etat du Totem","etat_totem")
    .withColumnRenamed("Nombre de vélo mécanique","nbre_velos_meca")
    .withColumnRenamed("Nombre vélo électrique","nbre_velos_elec")
    .withColumnRenamed("Code de la station","cod_station")
    .withColumnRenamed("Nom de la station","nom_station")
    .withColumnRenamed("Etat des stations","etat_station")
    .withColumnRenamed("Type de stations","type_station")
    .drop($"Nombre vélo en PARK+1")
    .drop($"PARK + activation")
    .drop($"Nombre vélo en PARK+14")
    .drop($"PARK +")
    .drop($"densityLevel")
    .drop($"description")
    .drop($"maxBikeOverflow")
    .drop($"nbFreeDock")
    .drop($"nbDock")

velibClean: org.apache.spark.sql.DataFrame = [nbre_bornes_dispo: int, nbre_bornes_station: int ... 12 more fields]


In [12]:
velibClean.printSchema()

root
 |-- nbre_bornes_dispo: integer (nullable = true)
 |-- nbre_bornes_station: integer (nullable = true)
 |-- achat_cb: string (nullable = true)
 |-- etat_totem: string (nullable = true)
 |-- nbre_velos_meca: integer (nullable = true)
 |-- nbre_velos_elec: integer (nullable = true)
 |-- cod_station: integer (nullable = true)
 |-- nom_station: string (nullable = true)
 |-- etat_station: string (nullable = true)
 |-- type_station: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- duedate: timestamp (nullable = true)
 |-- x_velib: double (nullable = true)
 |-- y_velib: double (nullable = true)



#### Compte tenu des puissances de calcul de mon ordinateur, je travaillerai uniquement sur les adresses du premier arrondissement de Paris 

In [13]:
val adresses01: DataFrame = adresses.filter($"code_postal" === 75001)

adresses01: org.apache.spark.sql.DataFrame = [id: string, id_fantoir: string ... 18 more fields]


In [14]:
val adresseClean: DataFrame =  adresses01
    .drop($"code_insee_ancienne_commune")
    .drop($"lon")
    .drop($"lat")
    .drop($"alias")
    .drop($"nom_ld")
    .drop($"libelle_acheminement")
    .drop($"nom_afnor")
    .drop($"source_position")
    .drop($"source_nom_voie")

adresseClean: org.apache.spark.sql.DataFrame = [id: string, id_fantoir: string ... 9 more fields]


In [15]:
adresseClean.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_fantoir: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- rep: string (nullable = true)
 |-- nom_voie: string (nullable = true)
 |-- code_postal: integer (nullable = true)
 |-- code_insee: integer (nullable = true)
 |-- nom_commune: string (nullable = true)
 |-- nom_ancienne_commune: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)



### 3- Recommandation de la station la plus proche de chaque adresse

In [16]:
val crossJoin = adresseClean.crossJoin(velibClean)

crossJoin: org.apache.spark.sql.DataFrame = [id: string, id_fantoir: string ... 23 more fields]


In [17]:
crossJoin.select($"numero", $"x", $"y",$"x_velib", $"y_velib").show(false)

+------+---------+----------+-------------+-------------+
|numero|x        |y         |x_velib      |y_velib      |
+------+---------+----------+-------------+-------------+
|1     |650852.89|6862571.29|646867.5476  |6863127.31356|
|1     |650852.89|6862571.29|651504.905142|6861726.97899|
|1     |650852.89|6862571.29|653410.789083|6863281.10064|
|1     |650852.89|6862571.29|651401.378236|6864567.90437|
|1     |650852.89|6862571.29|655430.484286|6861934.72171|
|1     |650852.89|6862571.29|655049.485913|6860263.89269|
|1     |650852.89|6862571.29|650472.822069|6861462.23007|
|1     |650852.89|6862571.29|651838.493023|6861475.44599|
|1     |650852.89|6862571.29|651784.241945|6857907.53976|
|1     |650852.89|6862571.29|648747.091267|6864888.51711|
|1     |650852.89|6862571.29|647760.557289|6864997.90998|
|1     |650852.89|6862571.29|653502.104984|6863633.0448 |
|1     |650852.89|6862571.29|650744.673053|6864031.55387|
|1     |650852.89|6862571.29|651265.173128|6859924.25257|
|1     |650852

In [18]:
val crossJoinDistInt: DataFrame = crossJoin
    .withColumn("dist_km", round(sqrt(pow($"x_velib" - $"x", 2) + pow($"y_velib" - $"y", 2))/1000, 3))
    .withColumn("proba",($"nbre_velos_meca"/ $"nbre_bornes_station"))

crossJoinDistInt: org.apache.spark.sql.DataFrame = [id: string, id_fantoir: string ... 25 more fields]


In [19]:
val crossJoinDist: DataFrame = crossJoinDistInt
    .withColumn("esperance", round(pow($"dist_km" + 1, -1) * $"proba" ,3))

crossJoinDist: org.apache.spark.sql.DataFrame = [id: string, id_fantoir: string ... 26 more fields]


In [20]:
val inter: DataFrame = crossJoinDist.groupBy("id").agg(min($"dist_km").as("dist_min"))

inter: org.apache.spark.sql.DataFrame = [id: string, dist_min: double]


In [21]:
val StationProche: DataFrame = inter.join(crossJoinDist, (crossJoinDist.col("id") ===  inter.col("id") && crossJoinDist.col("dist_km") === inter.col("dist_min")))

StationProche: org.apache.spark.sql.DataFrame = [id: string, dist_min: double ... 28 more fields]


In [22]:
StationProche.select($"numero", $"nom_voie", $"nom_station", $"cod_station", $"nbre_velos_meca", $"dist_km")
    .distinct().sort($"dist_km".asc).show(10,false)

+------+------------------------------------+-------------------------------------+-----------+---------------+-------+
|numero|nom_voie                            |nom_station                          |cod_station|nbre_velos_meca|dist_km|
+------+------------------------------------+-------------------------------------+-----------+---------------+-------+
|163   |Rue Saint-honore                    |Saint-Honoré - Musée du Louvre       |1023       |13             |0.003  |
|4     |Rue de Ventadour                    |Ventadour - Opéra                    |1116       |7              |0.004  |
|2     |Rue d'Alger                         |Alger - Rivoli                       |1018       |13             |0.004  |
|19    |Rue des Halles                      |Halles - Bourdonnais                 |1021       |6              |0.005  |
|217   |Rue Saint-honore                    |Saint-Honoré - 29 juillet            |1017       |13             |0.006  |
|4     |Rue de la Grande Truanderie     

### 4- Recommandation de la station ayant la plus forte espérance (Espérance basée sur la distance à la station et la probabilité d'avoir un vélo)

In [23]:
val esperances: DataFrame = crossJoinDist.groupBy("id").agg(max($"esperance").as("esperance_max"))

esperances: org.apache.spark.sql.DataFrame = [id: string, esperance_max: double]


In [24]:
val stationForteEsp: DataFrame = esperances.join(crossJoinDist, (crossJoinDist.col("id") ===  esperances.col("id") && crossJoinDist.col("esperance") === esperances.col("esperance_max")))

stationForteEsp: org.apache.spark.sql.DataFrame = [id: string, esperance_max: double ... 28 more fields]


In [25]:
stationForteEsp.select($"numero", $"nom_voie", $"nom_station", $"cod_station", $"nbre_velos_meca", $"dist_km", $"esperance")
    .distinct().sort($"esperance".desc).show(10,false)

+------+------------------+----------------------------+-----------+---------------+-------+---------+
|numero|nom_voie          |nom_station                 |cod_station|nbre_velos_meca|dist_km|esperance|
+------+------------------+----------------------------+-----------+---------------+-------+---------+
|37    |Rue Etienne Marcel|Etienne Marcel - Montorgueil|2002       |17             |0.01   |16.832   |
|35    |Rue Etienne Marcel|Etienne Marcel - Montorgueil|2002       |17             |0.019  |16.683   |
|29    |Rue Montorgueil   |Etienne Marcel - Montorgueil|2002       |17             |0.033  |16.457   |
|39    |Rue Etienne Marcel|Etienne Marcel - Montorgueil|2002       |17             |0.033  |16.457   |
|27    |Rue Montorgueil   |Etienne Marcel - Montorgueil|2002       |17             |0.038  |16.378   |
|25    |Rue Montorgueil   |Etienne Marcel - Montorgueil|2002       |17             |0.043  |16.299   |
|40    |Rue Montorgueil   |Etienne Marcel - Montorgueil|2002       |17   

In [101]:
//val url = "https://opendata.paris.fr/api/records/1.0/search/?dataset=velib-disponibilite-en-temps-reel&facet=overflowactivation&facet=creditcard&facet=kioskstate&facet=station_state" 
val url = "https://opendata.paris.fr/api/records/1.0/search/?dataset=velib-disponibilite-en-temps-reel&facet=records&rows=1391"
val data = scala.io.Source.fromURL(url).mkString

url: String = https://opendata.paris.fr/api/records/1.0/search/?dataset=velib-disponibilite-en-temps-reel&facet=records&rows=1391
data: String = {"nhits": 1391, "parameters": {"dataset": "velib-disponibilite-en-temps-reel", "timezone": "UTC", "rows": 1391, "format": "json", "facet": ["records"]}, "records": [{"datasetid": "velib-disponibilite-en-temps-reel", "recordid": "20e1277cebc85e0683be0b225f80798c4fb93f74", "fields": {"nbfreeedock": 31, "station_state": "Operative", "maxbikeoverflow": 0, "creditcard": "no", "station_type": "yes", "overflowactivation": "no", "station_code": "16107", "overflow": "no", "nbbikeoverflow": 0, "duedate": "2018-07-15", "densitylevel": "1", "nbedock": 35, "station": "{\"code\": \"16107\", \"name\": \"Benjamin Godard - Victor Hugo\", \"state\": \"Operative\...

In [102]:
val parsed = JSON.parseFull(data)

parsed: Option[Any] = Some(Map(nhits -> 1391.0, parameters -> Map(format -> json, timezone -> UTC, dataset -> velib-disponibilite-en-temps-reel, facet -> List(records), rows -> 1391.0), records -> List(Map(record_timestamp -> 2019-11-04T22:07:00.570000+00:00, fields -> Map(nbfreedock -> 0.0, station_type -> yes, overflow -> no, station_code -> 16107, station_name -> Benjamin Godard - Victor Hugo, nbfreeedock -> 31.0, duedate -> 2018-07-15, geo -> List(48.865983, 2.275725), nbdock -> 0.0, maxbikeoverflow -> 0.0, overflowactivation -> no, densitylevel -> 1, nbebike -> 2.0, kioskstate -> yes, station_state -> Operative, nbebikeoverflow -> 0.0, nbbikeoverflow -> 0.0, nbedock -> 35.0, creditcard -> no, station -> {"code": "16107", "name": "Benjamin Godard - Victor Hugo", "state": "Operative"...

In [103]:
val listAsAny = parsed match {
  case Some(e:Map[Any,Any]) => e("records")
  case None => println("Failed.")
}

<console>:40: warning: non-variable type argument Any in type pattern scala.collection.immutable.Map[Any,Any] (the underlying of Map[Any,Any]) is unchecked since it is eliminated by erasure
         case Some(e:Map[Any,Any]) => e("records")
                     ^
<console>:39: warning: match may not be exhaustive.
It would fail on the following input: Some((x: Any forSome x not in scala.collection.immutable.Map[?,?]))
       val listAsAny = parsed match {
                       ^
listAsAny: Any = List(Map(record_timestamp -> 2019-11-04T22:07:00.570000+00:00, fields -> Map(nbfreedock -> 0.0, station_type -> yes, overflow -> no, station_code -> 16107, station_name -> Benjamin Godard - Victor Hugo, nbfreeedock -> 31.0, duedate -> 2018-07-15, geo -> List(48.865983, 2.275725), nbdock -> 0.0, maxbikeoverflow -> 0.0, overflowactivation -> no, densitylevel -> 1, nbebike -> 2.0, kioskstate -> yes, station_state -> Operative, nbebikeoverflow -> 0.0, nbbikeoverflow -> 0.0, nbedock -> 35.0, credit

In [104]:
val recordsList = listAsAny.asInstanceOf[List[Map[String, Any]]]

recordsList: List[Map[String,Any]] = List(Map(record_timestamp -> 2019-11-04T22:07:00.570000+00:00, fields -> Map(nbfreedock -> 0.0, station_type -> yes, overflow -> no, station_code -> 16107, station_name -> Benjamin Godard - Victor Hugo, nbfreeedock -> 31.0, duedate -> 2018-07-15, geo -> List(48.865983, 2.275725), nbdock -> 0.0, maxbikeoverflow -> 0.0, overflowactivation -> no, densitylevel -> 1, nbebike -> 2.0, kioskstate -> yes, station_state -> Operative, nbebikeoverflow -> 0.0, nbbikeoverflow -> 0.0, nbedock -> 35.0, creditcard -> no, station -> {"code": "16107", "name": "Benjamin Godard - Victor Hugo", "state": "Operative", "type": "yes", "dueDate": 1531632033, "gps": {"latitude": 48.865983, "longitude": 2.275725}}, nbbike -> 2.0), recordid -> 20e1277cebc85e0683be0b225f80798c4fb9...

In [105]:
val recordsFields = recordsList.map(_("fields"))

recordsFields: List[Any] = List(Map(nbfreedock -> 0.0, station_type -> yes, overflow -> no, station_code -> 16107, station_name -> Benjamin Godard - Victor Hugo, nbfreeedock -> 31.0, duedate -> 2018-07-15, geo -> List(48.865983, 2.275725), nbdock -> 0.0, maxbikeoverflow -> 0.0, overflowactivation -> no, densitylevel -> 1, nbebike -> 2.0, kioskstate -> yes, station_state -> Operative, nbebikeoverflow -> 0.0, nbbikeoverflow -> 0.0, nbedock -> 35.0, creditcard -> no, station -> {"code": "16107", "name": "Benjamin Godard - Victor Hugo", "state": "Operative", "type": "yes", "dueDate": 1531632033, "gps": {"latitude": 48.865983, "longitude": 2.275725}}, nbbike -> 2.0), Map(nbfreedock -> 2.0, station_type -> yes, overflow -> no, station_code -> 6015, station_name -> André Mazet - Saint-André de...

In [106]:
val champs = recordsFields.asInstanceOf[List[Map[String, String]]]

champs: List[Map[String,String]] = List(Map(nbfreedock -> 0.0, station_type -> yes, overflow -> no, station_code -> 16107, station_name -> Benjamin Godard - Victor Hugo, nbfreeedock -> 31.0, duedate -> 2018-07-15, geo -> List(48.865983, 2.275725), nbdock -> 0.0, maxbikeoverflow -> 0.0, overflowactivation -> no, densitylevel -> 1, nbebike -> 2.0, kioskstate -> yes, station_state -> Operative, nbebikeoverflow -> 0.0, nbbikeoverflow -> 0.0, nbedock -> 35.0, creditcard -> no, station -> {"code": "16107", "name": "Benjamin Godard - Victor Hugo", "state": "Operative", "type": "yes", "dueDate": 1531632033, "gps": {"latitude": 48.865983, "longitude": 2.275725}}, nbbike -> 2.0), Map(nbfreedock -> 2.0, station_type -> yes, overflow -> no, station_code -> 6015, station_name -> André Mazet - Saint-...

In [127]:
val dfrdd = champs.map{x => (x("station_name"), x("station_code"), x("nbfreeedock"), x("nbedock"), x("nbfreedock"), x("nbbike"), x("nbdock"), x("nbebike"))}
val rdd = sc.makeRDD(dfrdd)

dfrdd: List[(String, String, String, String, String, String, String, String)] = List((Benjamin Godard - Victor Hugo,16107,31.0,35.0,0.0,2.0,0.0,2.0), (André Mazet - Saint-André des Arts,6015,40.0,52.0,2.0,4.0,3.0,8.0), (Faubourg Du Temple - Republique,11037,26.0,38.0,0.0,2.0,0.0,9.0), (Toudouze - Clauzel,9020,9.0,21.0,0.0,6.0,0.0,6.0), (Charonne - Robert et Sonia Delauney,11104,17.0,20.0,0.0,2.0,0.0,1.0), (Mairie du 12ème,12109,5.0,30.0,0.0,16.0,0.0,9.0), (Square Boucicaut,7003,41.0,60.0,0.0,15.0,0.0,3.0), (Harpe - Saint-Germain,5001,1.0,1.0,38.0,0.0,44.0,6.0), (Jourdan - Stade Charléty,14014,45.0,60.0,0.0,7.0,0.0,7.0), (Jouffroy d'Abbans - Wagram,17026,0.0,0.0,29.0,7.0,41.0,5.0), (Guersant - Gouvion-Saint-Cyr,17041,13.0,38.0,1.0,9.0,1.0,16.0), (Alibert - Jemmapes,10013,40.0,59.0,1.0,11...

In [124]:
val schema = new StructType()
  .add(StructField("nom_station", StringType, true))
  .add(StructField("cod_station", DoubleType, true))
  .add(StructField("nbre_bornes_dispo_elec", DoubleType, true))
  .add(StructField("nbre_bornes_station_elec", DoubleType, true))
  .add(StructField("nbre_bornes_dispo", DoubleType, true))
  .add(StructField("nbre_velos_meca", DoubleType, true))
  .add(StructField("nbre_bornes_station", DoubleType, true))
  .add(StructField("nbre_velos_elec", DoubleType, true))

schema: org.apache.spark.sql.types.StructType = StructType(StructField(nom_station,StringType,true), StructField(cod_station,DoubleType,true), StructField(nbre_bornes_dispo_elec,DoubleType,true), StructField(nbre_bornes_station_elec,DoubleType,true), StructField(nbre_bornes_dispo,DoubleType,true), StructField(nbre_velos_meca,DoubleType,true), StructField(nbre_bornes_station,DoubleType,true), StructField(nbre_velos_elec,DoubleType,true))


In [134]:
val velibTempReel = rdd.toDF("nom_station", "cod_station", "nbre_bornes_dispo_elec", "nbre_bornes_station_elec", "nbre_bornes_dispo", "nbre_velos_meca", "nbre_bornes_station", "nbre_velos_elec")

velibTempReel: org.apache.spark.sql.DataFrame = [nom_station: string, cod_station: string ... 6 more fields]


In [135]:
velibTempReel.printSchema()

root
 |-- nom_station: string (nullable = true)
 |-- cod_station: string (nullable = true)
 |-- nbre_bornes_dispo_elec: string (nullable = true)
 |-- nbre_bornes_station_elec: string (nullable = true)
 |-- nbre_bornes_dispo: string (nullable = true)
 |-- nbre_velos_meca: string (nullable = true)
 |-- nbre_bornes_station: string (nullable = true)
 |-- nbre_velos_elec: string (nullable = true)



In [136]:
println(s"Nombre de lignes : ${velibTempReel.count}")
println(s"Nombre de colonnes : ${velibTempReel.columns.length}")

Nombre de lignes : 1391
Nombre de colonnes : 8


In [137]:
velibTempReel.show()

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 48.0 failed 1 times, most recent failure: Lost task 0.0 in stage 48.0 (TID 1252, localhost, executor driver): java.lang.ClassCastException: java.lang.Double cannot be cast to java.lang.String

res51: Array[(String, String, String, String, String, String, String, String)] = Array((Benjamin Godard - Victor Hugo,16107,31.0,35.0,0.0,2.0,0.0,2.0), (André Mazet - Saint-André des Arts,6015,40.0,52.0,2.0,4.0,3.0,8.0), (Faubourg Du Temple - Republique,11037,26.0,38.0,0.0,2.0,0.0,9.0), (Toudouze - Clauzel,9020,9.0,21.0,0.0,6.0,0.0,6.0), (Charonne - Robert et Sonia Delauney,11104,17.0,20.0,0.0,2.0,0.0,1.0))


<console>: 42: warning: fruitless type test: a value of type (String, String, String, String, String, String, String, String) cannot also be a Array[T]